# Collaboration


- [Item 49: Write Docstrings for Every Function, Class, and Module](#Item-49:-Write-Docstrings-for-Every-Function,-Class,-and-Module)
- [Item 50: Use Packages to Organize Modules and Provide Stable APIs](#Item-50:-Use-Packages-to-Organize-Modules-and-Provide-Stable-APIs)
- [Item 51: Define a Root Exception to Insulate Callers from APIs](#Item-51:-Define-a-Root-Exception-to-Insulate-Callers-from-APIs)
- [Item 52: Know How to Break Circular Dependencies](#Item-52:-Know-How-to-Break-Circular-Dependencies)

## Item 49: Write Docstrings for Every Function, Class, and Module

In [ ]:
"""
Documentation from a program's source code is
directly accessible as the program runs
"""

def palindrome(word):
    """Return True if the given word"""
    return word == word[::-1]

print(repr(palindrome.__doc__))
help(palindrome)

### Documenting Modules

Each module should have a top-level docstring. This is a string literal that is the first statement in a source file. It should use three double quotes ("""). The goal of this docstring is to introduce the module and its contents.

The first line of the docstring should be a single sentence describing the module's purpose. The paragraphs that follow should contain the details that all users of the module should know about its operation. The module docstring is also a jumping-off point where you can highlight important classes and functions found in the module.

Example below:

In [ ]:
# words.py
# !/usr/bin/env python3
"""Library for testing words for various linguistic patterns.

Testing how words relate to each other can be tricky sometimes! This module provides easy ways to determine when words you’ve found have special properties.

Available functions:
- palindrome: Determine if a word is a palindrome.
- check_anagram: Determine if two words are anagrams.
...
"""

# ...

### Documenting Classes

In [ ]:
class Player(object):
    """Represents a player of the game.
    
    Subclasses may override the 'tick' method to provide
    custom animations for the player's movement depending
    on their power level, etc.
    
    Public attributes:
    - power: Unused power-ups (float between 0 and 1).
    - coins: Coins found during the level (integer).
    """
    
    # ...

### Documenting Functions

Any return values should be mentioned. Any exceptions that callers must handle as part of the function's interface should be explained.

In [ ]:
def find_anagrams(word, dictionary):
    """Find all anagrams for a word.
    
    This function only runs as fast as the test for
    membership in the 'dictionary' container. It will
    be slow if the dictionary is a list and fast if
    it's a set.
    
    Args:
        word: String of the target word.
        dictionary: Container with all strings that
            are known to be actual words.
            
    Return:
        List of anagrams that were found. Empty if
        none were found.
    """
    
    # ...

### Things to Remember

- Write documentation for every module, class, and function using docstrings. Keep them up to date as your code changes.
- For modules: Introduce the contents of the module and any important classes or functions all users should know about.
- For classes: Document behavior, important attributes, and subclass behavior in the docstring following the *class* statement.
- For functions and methods: Document every argument, returned value, raised exception, adn other behaviors in the docstring following the *def* statement.

## Item 50: Use Packages to Organize Modules and Provide Stable APIs

### Namespaces

```python
from analysis.utils import inspect as analysis_inspect
from frontend.utils import inspect as frontend_inspect
```

### Stable APIs

Python can limit the surface area exposed to API consumers by using the \_\_all__ special attribute of a module or package. The value of \_\_all__ is a list of every name to export from the module as part of its public API.

```python
# models.py
__all__ = ['Projectile']

class Projectile(object):
    def __init__(self, mass, velocity):
        self.mass = mass
        self.velocity = velocity
        
# utils.py
from . models import Projectile

__all__ = ['simulate_collision']

def _dot_product(a, b):
    # ...
    
def simulate_collision(a, b):
    # ...
    
# __init__.py
__all__ = []
from . models import *
__all__ += models.__all__
from . utils import *
__all__ += utils.__all__

```

### Things to Remember

- Packages in Python are modules that contain other modules. Packages allow you to organize your code into separate, non-conflicting namespaces with unique absolute module names.
- Simple packages are defined by adding an \_\_init__.py file to a directory that contains other source files. These files become the child modules of the directory's package. Package directories may also contain other packages.
- You can provide an explicit API for a module by listing its publicly visible names in its \_\_all__ special attribute.
- You can hide a packages's internal implementation by only importing public names in the package's \_\_init__.py file or by naming internal-only members with a leading underscore.
- When collaborating within a single team or on a single codebase, using \_\_all__ for explicit APIs is probably unnecessary.

## Item 51: Define a Root Exception to Insulate Callers from APIs

When you're defining a module's API, the exceptions you throw are just as much a part of your inteface as the functions and classes you define.

```python
# my_module.py
class Error(Exception):
    """Base-class for all exceptions raised by this module."""

    
class InvalidDensityError(Error):
    """There was a problem with a provided density value."""
    
    
def determine_weight(volume, density):
    if density <= 0:
        raise ValueError('Density must be positive')
    # ...
    
try:
    weight = my_module.determine_weight(1, -1)
except my_module.Error as e:
    logging.error('Unexpected error: %s', e)
except Exception as e:
    logging.error('Bug in the API code: %s', e)
    raise
```

This insulation has three helpful effects:

1. Root exceptions let callers understand when there's a problem with their usage of your API.
2. Root exceptions can help find bugs in your API module's code. If your code only deliberately raises exceptions taht you define within your module's hierarchy, then all other types of exceptions raised by your module must be the ones that you didn't intend to raise. These are bugs in your API's code.
3. The third impact of using root exceptions is future-proofing your API. Over time, you may want to expand your API to provide more specific exceptions in certain situations.

### Things to Remember

- Defining root exceptions for your modules allows API consumers to insulate themselves from your API.
- Catching root exceptions can help you find bugs in code that consumes an API.
- Catching the Python *Exception* base class can help you find bugs in API implementations.
- Intermediate root exceptions let you add more specific types of exceptions in the future without breaking your API consumers.

## Item 52: Know How to Break Circular Dependencies

When a module is imported, here's what Python actually does in depth-first order:

1. Searches for your module in locations from sys.path
2. Loads the code from the module and ensures that it compiles
3. Creates a corresponding empty module object
4. Inserts the module into sys.modules
5. Runs the code in the module object to define its contents

Three ways to break circular dependencies:

1. Reordering Imports

```python
#app.py
class Prefs(object):
    # ...
    
prefs = Prefs()

import dialog  # Moved
dialog.show()
```

Although this avoids the *AttributeError*, it goes against the PEP 8 style guide. The style guide suggests that you always put imports at the top of your Python files.

Having imports later in a file can be brittle and can cause small changes in the ordering of your code to break the module entirely. Thus, you should avoid import reordering to solve your circular dependency issues.

2. Import, Configure, Run

The second solution to the circular imports problem is to have your modules minimize side effects at import time. You have your modules only define functions, classes, and constants. You avoid actually running any functions at import time. Then, you have each module provide a *configure* function that you call once all other modules have finished importing. The purpose of *configure* is to prepare each module's state by accessing the attributes of other modules. You run *configure* after all modules have been imported, so all attributes must be defined.

```python
# dialog.py
import app

class Dialog(object):
    # ...
    
save_dialog = Dialog()

def show():
    # ...
    
def configure():
    save_dialog.save_dir = app.prefs.get('save_dir')
 

# app.py
import dialog

class Prefs(object):
    # ...
    
prefs = Prefs()

def configure():
    # ...
    
    
# main.py
import app
import dialog

app.configure()
dialog.configure()

dialog.show()
```

This works well in many situations and enables patterns like *dependency injection*. But sometimes it can be difficult to structure your code so that an explicit *configure* step is possible. Having two distinct phases within a module can also make your code harder to read because it separates the definition of objects from their configuration.

3. Dynamic Import

The third --- and often simplest --- solution to the circular imports problem is to use an *import* statement within a function or method. This is called a *dynamic import* because the module import happens while the program is running, not while the program is first starting up and initializing its modules.

```python
# dialog.py
class Dialog(object):
    # ...
    
save_dialog = Dialog()

def show():
    import app  # Dynamic import
    save_dialog.save_dir = app.prefs.get('save_dir')
    # ...
```

In general, it's good to avoid dynamic imports like this. The cost of the *import* statement is not negligible and can be especially bad in tight loops. However, these downsides are often better than the alternative of restructuring your entire program.

### Things to Remember

- Circular dependencies happen when two modules must call into each other at import time. They can cause your program to crash at startup.
- The best way to break a circular dependency is refactoring mutual dependencies into a separate module at the bottom of the dependency tree.
- Dynamic imports are the simplest solution for breaking a circular dependency between modules while minimizing refactoring and complexity.